In [33]:
#PARAMETERS
testDataFile="/Users/eliotpbrenner/PycharmProjects/SparsityBoost/data/synthetic_examples/experiments/0/alarm1000.dat"
numTestVariable=37
numTestDataPts=1000
studentDataFile="student_100kSamples.csv"
studentNValsFile="student_nvals.csv"
numStudentVariable=5
numStudentDataPts=100000

100000

In [2]:
Pkg.update()

INFO: Updating METADATA...
INFO: Computing changes...
INFO: No packages to install, update or remove


In [3]:
# Imports
using Base.Test
using Iterators

In [4]:
type model
  nodes::Int64
  nval::Array{Int64,1}
  dataArray::Array{Int64,2}
end

In [5]:
type jointDistWithMarginals
  pdist::Array{Float64,2}
  p_A::Array{Float64,1}
  p_B::Array{Float64,1}
end

In [6]:
# Constructor which calculates the marginals
jointDistWithMarginals(pdist::Array{Float64,2}) = jointDistWithMarginals(pdist, vec(sum(pdist,1)), vec(sum(pdist,2)))

jointDistWithMarginals

In [7]:
jointDistWithMarginals([[0.25 0.25]; [ 0.25 0.25]])

jointDistWithMarginals(2x2 Array{Float64,2}:
 0.25  0.25
 0.25  0.25,[0.5,0.5],[0.5,0.5])

In [8]:
function readDataBinary(filePath::AbstractString)
  dataArray = readdlm(filePath, ' ', Int);
  nodes=size(dataArray)[2];
  nval=2*ones(Int, nodes)
  theModel = model(nodes,nval,dataArray);
end


readDataBinary (generic function with 1 method)

In [29]:
function readData(dataFilePath::AbstractString, nvalsFilePath::AbstractString)
  dataArray = readdlm(dataFilePath, ',', Int);
  nodes=size(dataArray)[2];
    nval=vec(readdlm(nvalsFilePath, ',', Int));   #2*ones(Int, nodes)
  theModel = model(nodes,nval,dataArray);
end

readData (generic function with 1 method)

In [31]:
m=readData(studentDataFile, studentNValsFile)

model(5,[2,2,3,2,2],100000x5 Array{Int64,2}:
 0  0  1  0  1
 0  0  0  0  1
 0  0  0  0  1
 0  0  2  0  0
 1  1  1  1  0
 1  1  0  1  1
 0  1  0  1  1
 0  0  2  0  0
 1  1  1  1  1
 1  0  2  0  0
 1  0  2  1  0
 0  1  0  1  1
 0  0  2  0  0
 ⋮            
 0  0  0  0  1
 0  0  0  0  1
 1  0  2  0  0
 0  0  1  0  1
 1  0  2  0  0
 1  1  0  1  1
 0  0  1  0  0
 1  1  2  0  0
 1  0  2  0  0
 1  1  0  1  1
 1  0  2  0  0
 0  0  1  0  1)

In [35]:
# test readData function
m=readData(studentDataFile, studentNValsFile)
@test size(m.dataArray) == (numStudentDataPts, numStudentVariable)

In [36]:
function verify(aJointDistributionWithMarginals::jointDistWithMarginals)
  sumA=sum(aJointDistributionWithMarginals.pdist,1)
  sumB=sum(aJointDistributionWithMarginals.pdist,2)
  @test_approx_eq(transpose(sumA),aJointDistributionWithMarginals.p_A)
  @test_approx_eq(sumB,aJointDistributionWithMarginals.p_B)
  @test_approx_eq(sum(sumA),1.0)
  return true
end

verify (generic function with 1 method)

In [37]:
goodDist=jointDistWithMarginals([[0.25 0.25]; [ 0.25 0.25]], [0.5; 0.5], [0.5; 0.5])
@test verify(goodDist) == true

In [38]:
#TODO: determine why this this test fails instead of succeeding!
badDist = jointDistWithMarginals([[0.25 0.25]; [ 0.25 0.25]], [0.6; 0.4], [0.5; 0.5])
@test_throws( ErrorException, verify(badDist))

ErrorException("assertion failed: |transpose(sumA) - aJointDistributionWithMarginals.p_A| <= 2.220446049250313e-12\n  transpose(sumA) = [0.5\n 0.5]\n  aJointDistributionWithMarginals.p_A = [0.6,0.4]\n  difference = 0.09999999999999998 > 2.220446049250313e-12")

In [39]:
function MI_term(i::Int, j::Int, dist::jointDistWithMarginals)
  return dist.pdist[i,j]*log(dist.pdist[i,j]/(dist.p_A[i]*dist.p_B[j])  )
end

MI_term (generic function with 1 method)

In [40]:
function MI(dist::jointDistWithMarginals)
  (a,b)=size(dist.pdist)
  return sum([(MI_term(i,j,dist)) for i=1:a, j=1:b])
end

MI (generic function with 1 method)

In [41]:
MI_term(1,1,goodDist)

0.0

In [42]:
@test_approx_eq(MI(goodDist),0)

In [46]:
t=0.01
anotherDist=jointDistWithMarginals([[0.25+t 0.25-t]; [ 0.25-t 0.25+t]], [0.5; 0.5], [0.5; 0.5])
verify(anotherDist)
@test_approx_eq(MI(anotherDist), 0.0008002134699838133)

In [47]:
testNotInDict(elt::Int, SToVal::Dict) = try
    """TODO: figure out how to propagate y up the call stack if it's not a KeyError"""
    x=SToVal[elt] 
    warn("$elt is in $SToVal with value $x")
    throw(DomainError())
  catch y
    if isa(y, KeyError)
      return true
    else
      return y
    end
  end

testNotInDict (generic function with 1 method)

In [48]:
S=Dict(1=>2, 3=>4)

Dict{Int64,Int64} with 2 entries:
  3 => 4
  1 => 2

In [49]:
testNotInDict(1,S)

DomainError()

In [50]:
function conditionalDist(m::model,i::Int,j::Int,SToVal::Dict)
  """
  TODO: that S is a dict of integers and that both are valid for the model
  TODO: vectorize the computation of the pDist
  """
  @test i!=j
  @test testNotInDict(i, SToVal) == true
  @test testNotInDict(j, SToVal) ==true
  selectedPoints = m.dataArray[reduce(&,[m.dataArray[:,condVar] .==SToVal[condVar] for condVar in keys(SToVal)]),:]
  numSelectedPoints = size(selectedPoints)[1]
  pDist = zeros(m.nval[i], m.nval[j])
  for k = 1:m.nval[i]
      for l = 1:m.nval[j]
          pDist[k,l] = size(selectedPoints[(selectedPoints[:,i] .==k-1)&(selectedPoints[:,j] .==l-1), :])[1]/numSelectedPoints
    end
  end
    jd = jointDistWithMarginals(pDist)
    verify(jd)
    return jd
end

conditionalDist (generic function with 1 method)

In [51]:
SToVal = Dict(3=>0, 4=>1)
cd = conditionalDist(m, 1, 2, SToVal)

jointDistWithMarginals(2x2 Array{Float64,2}:
 0.0336934   0.703316
 0.00365415  0.259337,[0.03734752001719598,0.962652479982804],[0.7370089741522919,0.2629910258477081])

In [52]:
@test_throws(ErrorException,conditionalDist(m, 1, 1, SToVal))

ErrorException("test failed: 1 != 1\n in expression: i != j")

In [53]:
conditionalDist(m, 3, 1, SToVal)

LoadError: LoadError: test failed: DomainError() == true
 in expression: testNotInDict(i,SToVal) == true
while loading In[53], in expression starting on line 1

In [54]:
#TODO: determine why this test is failing
@test_throws(DomainError, conditionalDist(m, 3, 1, S))

LoadError: LoadError: test failed: ErrorException("test failed: DomainError() == true\n in expression: testNotInDict(i,SToVal) == true") was thrown instead of DomainError
 in expression: conditionalDist(m,3,1,S)
while loading In[54], in expression starting on line 2

In [55]:
conditionalDist(m, 3, 1, S)

LoadError: LoadError: test failed: DomainError() == true
 in expression: testNotInDict(i,SToVal) == true
while loading In[55], in expression starting on line 1

In [56]:
function MI(m::model,i::Int,j::Int,SToVal::Dict)
  """
  TODO: test
  """
  MI(conditionalDist(m::model,i::Int,j::Int,SToVal::Dict))
end

MI (generic function with 2 methods)

In [70]:
function MI(m::model, i::Int, j::Int, S::Array{Int64,1})
  """
  TODO: test
  """
  max=-Inf
  for p in Iterators.product([1:m.nval[i]-1 for i in S]...)
        candidate=MI(m,i,j, Dict(zip(S,p)))
    if candidate > max
      max=candidate
    end
  end
  return max
end

MI (generic function with 3 methods)

In [64]:
#MI(m,1,2, Dict(3=>1, 7=>1, 22=>1))
MI(m,1,2,Dict(3=>1, 4=>1, 5=>1))

0.15380726028635627

In [88]:
function epsilon(m::model, i::Int, j::Int, d::Int)
    min=Inf
    bestK=Array([])
    for di in (1:d)
        for k in combinations(1:m.nodes,di)
            if (i in k) || (j in k)
                continue
            end
            candidate = MI(m,i,j,k)
            if candidate < min
                min=candidate
                bestK=k
            end
        end
    end
    return bestK, min
end

epsilon (generic function with 1 method)

In [89]:
epsilon(m,1,2,1)

([5],0.18222429967200127)

In [ ]:
using PyCall
@pyimport pylab
x = linspace(0,2*pi,1000); y = sin(3*x + 4*cos(2*x));
pylab.plot(x, y; color="red", linewidth=2.0, linestyle="--")
pylab.show() 

In [ ]:
Pkg.add("PyCall")

In [91]:
using PyCall


In [92]:
@pyimport graphviz


In [93]:
@pyimport libpgm

In [121]:
@pyimport libpgm.nodedata as nd
@pyimport libpgm.discretebayesiannetwork as dbn
@pyimport libpgm.graphskeleton as gs
@pyimport libpgm.tablecpdfactorization as tf

In [123]:
nodedata = nd.NodeData().load(jsonpath)
skel = gs.GraphSkeleton()

LoadError: LoadError: type PyObject has no field load
while loading In[123], in expression starting on line 1

In [105]:
jsonpath="student.txt"
nodedata.load(jsonpath)
skel.load(jsonpath)
load bayesian network
bn = dbn(skel, nd)
tablecpd=tf.TableCPDFactorization(bn)

LoadError: LoadError: PyError (:Py_CompileString) <type 'exceptions.SyntaxError'>
SyntaxError('invalid syntax', ('PyCall.jl', 1, 10, 'nodedata = nd.NodeData()'))

while loading In[105], in expression starting on line 1

In [120]:
nodedata

PyObject <libpgm.nodedata.NodeData object at 0x31f30d590>

In [127]:
nda=pyimport("libpgm")["nodedata"]

PyObject <module 'libpgm.nodedata' from '/Users/eliotpbrenner/anaconda/lib/python2.7/site-packages/libpgm/nodedata.pyc'>

In [137]:
ndb=pycall(nda["NodeData"], PyCall.PyObject)

PyObject <libpgm.nodedata.NodeData object at 0x31f30d2d0>

In [142]:
pycall(ndb["load"],PyCall.PyObject, jsonpath)

PyObject None

In [143]:
skel = gs.GraphSkeleton()

PyObject <libpgm.graphskeleton.GraphSkeleton object at 0x31f30d4d0>

In [144]:
pycall(skel["load"], PyCall.PyObject, jsonpath)

PyObject None

In [152]:
dbna = pyimport("libpgm")["discretebayesiannetwork"]

PyObject <module 'libpgm.discretebayesiannetwork' from '/Users/eliotpbrenner/anaconda/lib/python2.7/site-packages/libpgm/discretebayesiannetwork.pyc'>

In [156]:
adbn = pycall(dbna["DiscreteBayesianNetwork"], PyCall.PyObject, skel, ndb)

PyObject <libpgm.discretebayesiannetwork.DiscreteBayesianNetwork object at 0x31f30d610>

In [161]:
typeof(adbn)

PyCall.PyObject

In [168]:
pycall(adbn["__getattribute__"], Array, "V")

5-element Array{Any,1}:
 "Difficulty"  
 "Intelligence"
 "Grade"       
 "SAT"         
 "Letter"      